In [1]:
from typing import Set, Any, Union, List, Dict, Tuple, Optional

import pandas as pd
import numpy as np

In [2]:
courses = pd.read_csv("../data/courses.csv")
students = pd.read_csv("../data/students.csv")

# Merge all courses of each student into a single column of lists
students["cx"] = students.apply(
    lambda row: [row[c] for c in students if "c" in c and isinstance(row[c], str)],
    axis=1,
)

In [3]:
class Course:
    courses = set()

    def __init__(
        self, name: str, credit_hours: int, avg_grade: float, avg_coursework: float
    ):
        self.name = name
        self.students = students
        self.credit_hours = credit_hours
        self.avg_grade = avg_grade
        self.avg_coursework = avg_coursework

        if self not in self.courses:
            self.courses.add(self)

    @classmethod
    def get_course_by_name(cls, name: str):
        for course in cls.courses:
            if course.name == name:
                return course
        return None

    @classmethod
    def from_row(cls, row: pd.Series):
        return Course(row["name"], row["ch"], row["grade"], row["coursework"])

    def __repr__(self):
        return f"Course(name={self.name}, ch={self.credit_hours}, grade={self.avg_grade}, avg_cw={self.avg_coursework})"

    def __eq__(self, other):
        return isinstance(other, Course) and self.name == other.name

    def __hash__(self):
        return hash(self.name)


class Student:
    students = set()

    def __init__(self, name: str, courses: Set[Course]):
        self.name = name
        self.courses = courses
        if self not in self.students:
            self.students.add(self)

    def add_course(self, course: Course):
        self.courses.add(course)

    @classmethod
    def from_row(cls, row: pd.Series):
        courses = {
            Course.get_course_by_name(c)
            for c in row["cx"]
            if Course.get_course_by_name(c) is not None
        }

        return Student(row["name"], courses)

    def __repr__(self):
        return f"Student(name={self.name}, courses={self.courses})"

    def __str__(self):
        courses = ",".join([c.name for c in self.courses])
        return f"Student(name={self.name}, courses={courses})"

    def __eq__(self, other):
        return isinstance(other, Student) and self.name == other.name

    def __hash__(self):
        return hash(self.name)

In [4]:
courses.apply(lambda row: Course.from_row(row), axis=1)
students.apply(lambda row: Student.from_row(row), axis=1)

None

In [5]:
for s in Student.students:
    print(s)

Student(name=Ashour, courses=SGN,NTW,RTS,DSA,EDA,MLE)
Student(name=Abdo, courses=NTW,RTS,DSA,EMB,EDA,MLE)
Student(name=Obai, courses=SGN,DLE,RTS,DSA,EMB,MLE,AGL)
Student(name=Megh, courses=NTW,DLE,RTS,DSA,EMB,EDA,AGL)
Student(name=Ziad, courses=SGN,NTW,DSA,EMB,MTH,EDA,MLE)
Student(name=Kareem, courses=NTW,DLE,RTS,MTH,MLE,AGL)
Student(name=Osama, courses=SGN,NTW,DLE,RTS,MTH,MLE)


In [7]:
class TimeTable:
    def __init__(self, courses: Set[Course], students: Set[Student]):
        self.courses = courses
        self.students = students

    def course_frequency(self, course: Course):
        return len([s for s in self.students if course in s.courses])

    def __repr__(self):
        return f"TimeTable(courses={self.courses}, students={self.students})"

    def __str__(self):
        courses = ",".join([course.name for course in self.courses])
        students = ",".join([student.name for student in self.students])
        return f"TimeTable(courses={courses}, students={students})"


timetable = TimeTable(Course.courses, Student.students)
print(timetable)

TimeTable(courses=SGN,NTW,DLE,RTS,DSA,EMB,MTH,EDA,MLE,AGL, students=Ashour,Abdo,Obai,Megh,Ziad,Kareem,Osama)


In [10]:
SGN = Course.get_course_by_name("SGN")
NTW = Course.get_course_by_name("NTW")

In [12]:
timetable.course_frequency(NTW)

6